In [1]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [2]:
# %cd /content/drive/Othercomputers/Lenovo/BINUS/SKRIPSI/PRE THESIS/CODES

%cd /content/drive/MyDrive/CODES

/content/drive/.shortcut-targets-by-id/17Ue_9vMIt3n5ER_-PQnFm3xwK8yDUQYX/CODES


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls

 01b_bertopic_cleaning_eda_preprocessing.ipynb
 01_cleansing_and_preprocessing.ipynb
 01c_sentiment_prediction_pipeline
 02_modeling_sentiment_prediction.ipynb
 03_modeling_topic_modeling.ipynb
 data_preparation.py
 df_modeling_BERT.csv
 df_modeling_bertopic.csv
 df_modeling.csv
 df_test2.csv
 df_test.csv
 df_train2.csv
 df_train.csv
 df_val.csv
 exclude_words.py
 mapping_dictionary.py
 negative_topic_modeling_results.xlsx
'Normalization Dictionary [Explained].gsheet'
 normalization_dictionary.py
 positive_topic_modeling_results.xlsx
 __pycache__
 results
 src
 stopwords.txt
 tes-bertopic.ipynb


# **Import Libraries and Prepare Dataset**

In [8]:
pip install -q bertopic

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nlp-id 0.1.19.0 requires huggingface-hub==0.31.1, but you have huggingface-hub 0.34.3 which is incompatible.


In [ ]:
pip install -q nlp_id

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.54.1 requires huggingface-hub<1.0,>=0.34.0, but you have huggingface-hub 0.31.1 which is incompatible.


In [4]:
import pandas as pd
import numpy as np
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
import hdbscan
import re

import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)

In [5]:
df_modeling = pd.read_csv('df_modeling_BERT.csv')
df_modeling['sentiment'] = df_modeling['sentiment'].map({'positive': 1, 'negative': 0})
df_modeling.head()

,cleaned_text,sentiment
0,saya suka materi yang sudah disiapkan oleh pihak kampus karena memudahkan mahasiswa saya juga menyukai program enrichment yang disediakan kampus saya sehingga mahasiswa dapat belajar di ruang lingkup yang lebih luas,1
1,bisa bertemu dengan teman teman baru dan mendapatkan koneksi serta mendapatkan pelajaran yang berguna bagi saya kedepan nya,1
2,saya suka dengan makanan yang ada di dalam kampus saya terutama bakmi efata selain itu disekitar kampus juga banyak makanan enak,1
3,fasilitas kampus alam sutera sangat bagus pelajaran lab diajarkan oleh asisten yang sangat mengerti materi,1
4,saya suka dengan pertemanan nya solid mau saling bantu satu sama lain bagi bagi kisi kisi pas ujian terus saling ngajarin,1


In [6]:
df_pos = df_modeling[df_modeling['sentiment'] == 1]
df_neg = df_modeling[df_modeling['sentiment'] == 0]

In [7]:
def clean_text(text, negation=True):
  if negation:
      for phrase in ['sangat tidak menyukai', 'tidak menyukai','sangat tidak suka', 'tidak suka', 'kurang suka', 'kurang menyukai', 'ga suka', 'gak suka', 'ga menyukai', 'gak menyukai']:
          text = text.replace(phrase, '')

  for phrase in ['suka', 'sangat suka', 'menyukai', 'sangat menyukai']:
      text = text.replace(phrase, '')

  text = re.sub(r'\s+', ' ', text).strip()
  return text

In [8]:
from exclude_words import exclude_stopwords
from nlp_id.stopword import StopWord
from nlp_id.tokenizer import Tokenizer

stopword = StopWord()
tokenizer = Tokenizer()

# Menambahkan kata untuk stop words
stop_words = stopword.get_stopword()
custom_stopwords = ['nya', 'ya', 'nih']
stop_words.append(custom_stopwords)

def text_preprocessing(text):

  # # Tokenisasi menggunakan Tokenizer dari nlp_id
  tokens = tokenizer.tokenize(text)

  filtered_tokens = [word for word in tokens if word not in stop_words]

  # Menghapus spasi yang berlebih
  text = re.sub(r'\s+', ' ', text).strip()

  processed_text = " ".join(filtered_tokens)

  return processed_text

In [9]:
df_pos['cleaned_text'] = df_pos['cleaned_text'].apply(clean_text, negation=False)
df_neg['cleaned_text'] = df_neg['cleaned_text'].apply(clean_text, negation=True)
df_pos['cleaned_text'] = df_pos['cleaned_text'].apply(text_preprocessing)
df_neg['cleaned_text'] = df_neg['cleaned_text'].apply(text_preprocessing)

In [10]:
texts_pos = df_pos["cleaned_text"].astype(str).tolist()
texts_neg = df_neg["cleaned_text"].astype(str).tolist()

# **BERTopic**

### Create Model

In [ ]:
embedding_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

topic_model_pos = BERTopic(embedding_model=embedding_model, verbose=True)
topic_model_neg = BERTopic(embedding_model=embedding_model, verbose=True)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Fit Model

In [ ]:
topics_pos, probs_pos = topic_model_pos.fit_transform(texts_pos)

2025-07-16 15:02:44,255 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/10 [00:00<?, ?it/s]

2025-07-16 15:02:45,176 - BERTopic - Embedding - Completed ✓
2025-07-16 15:02:45,177 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-07-16 15:02:53,085 - BERTopic - Dimensionality - Completed ✓
2025-07-16 15:02:53,086 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-07-16 15:02:53,102 - BERTopic - Cluster - Completed ✓
2025-07-16 15:02:53,106 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-07-16 15:02:53,128 - BERTopic - Representation - Completed ✓


In [ ]:
topics_neg, probs_neg = topic_model_neg.fit_transform(texts_neg)

2025-07-16 15:02:53,149 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/10 [00:00<?, ?it/s]

2025-07-16 15:02:53,458 - BERTopic - Embedding - Completed ✓
2025-07-16 15:02:53,459 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-07-16 15:02:54,210 - BERTopic - Dimensionality - Completed ✓
2025-07-16 15:02:54,211 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-07-16 15:02:54,227 - BERTopic - Cluster - Completed ✓
2025-07-16 15:02:54,230 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-07-16 15:02:54,247 - BERTopic - Representation - Completed ✓


### Inspect Topics

In [ ]:
topic_model_pos.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,289,0_teman_kampus_dosen_materi,"[teman, kampus, dosen, materi, belajar, perkuliahan, lingkungan, kuliah, mahasiswa, hal]","[dosen dosen kampus friendly materi mudah dipelajari teman teman pelajaran perkuliahan, perkuliahan lingkungan kampus nyaman kondusif belajar teman teman suportif aktif kegiatan pengalaman kuliah menyenangkan, teman teman mendukung membantu perkuliahan teman teman belajar bareng memahami materi]"
1,1,12,1_makanan_kampus_kantin_fasilitas,"[makanan, kampus, kantin, fasilitas, teman, area, makan, peluang, enak, pilihan]","[makanan fasilitas wilayah kampus kampus terpenuhi, daerah kampus makanan explore makanan jajan teman teman terdekat seru gak nyangka circle real sma, makanan kampus bakmi efata disekitar kampus makanan enak]"


In [ ]:
topic_model_neg.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,188,0_dosen_kampus_mahasiswa_kelas,"[dosen, kampus, mahasiswa, kelas, kuliah, materi, hal, fasilitas, tugas, perkuliahan]","[perkuliahan hal i jadwal perkuliahan padat tugas tugas menumpuk bersamaan hal sulit membagi optimal mata kuliah penyampaian materi menarik aplikatif proses belajar menyenangkan berusaha mengambil pelajaran pengalaman, hal i kuliah dosen dosen membawakan materi interaksi terbatas mahasiswa interaksi terbatas mahasiswa cepat bosan memperhatikan materi pembelajaran, fasilitas mahasiswa tugas kelas berukuran besar mengurangi interaksi dosen mahasiswa]"
1,1,65,1_toilet_fasilitas_tisu_kampus,"[toilet, fasilitas, tisu, kampus, wifi, bersih, tissue, nyaman, gedung, lambat]","[fasilitas toilet bersih tisu mencukupi wifi terkoneksi tugas lancar kampus, fasilitas toilet bersih tisu mencukupi wifi terkoneksi tugas lancar kampus, fasilitas toilet bersih tisu mencukupi wifi terkoneksi tugas lancar kampus]"
2,2,48,2_wifi_kampus_koneksi_susah,"[wifi, kampus, koneksi, susah, absensi, fasilitas, internet, jaringan, terkoneksi, lambat]","[fasilitas kampus wifi terputus koneksi jaringan lancar, wifi disediakan kampus koneksi jaringan kesulitan tugas absen, koneksi jaringan wifi kampus wifi terkonek aktivitas kampus kesulitan termaksa memakai data hotspot teman aktivitas krusial absensi kelas wifi]"


### Visualize Topic

In [ ]:
topic_model_pos.visualize_barchart()

In [ ]:
topic_model_neg.visualize_barchart()

In [ ]:
# topic_model_neg.visualize_topics()

In [ ]:
topic_model_pos.visualize_documents(texts_pos)

In [ ]:
topic_model_neg.visualize_documents(texts_neg)

# **TUNING**

In [ ]:
umap_model = UMAP(n_neighbors=7, min_dist=0.0, metric="cosine")
hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=5, min_samples=None, cluster_selection_epsilon=0.0)
embedding_model = SentenceTransformer("indobenchmark/indobert-base-p1")

topic_model_pos_tuned = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    verbose=True
)

topic_model_neg_tuned = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    verbose=True
)

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
topics_pos_tuned, probs_pos_tuned = topic_model_pos_tuned.fit_transform(texts_pos)

2025-07-16 15:14:44,544 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/10 [00:00<?, ?it/s]

2025-07-16 15:14:45,318 - BERTopic - Embedding - Completed ✓
2025-07-16 15:14:45,324 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-07-16 15:14:46,112 - BERTopic - Dimensionality - Completed ✓
2025-07-16 15:14:46,113 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-07-16 15:14:46,128 - BERTopic - Cluster - Completed ✓
2025-07-16 15:14:46,135 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-07-16 15:14:46,177 - BERTopic - Representation - Completed ✓


In [ ]:
topics_neg_tuned, probs_neg_tuned = topic_model_neg_tuned.fit_transform(texts_neg)

2025-07-16 15:14:46,243 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/10 [00:00<?, ?it/s]

2025-07-16 15:14:46,901 - BERTopic - Embedding - Completed ✓
2025-07-16 15:14:46,902 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-07-16 15:14:47,273 - BERTopic - Dimensionality - Completed ✓
2025-07-16 15:14:47,274 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-07-16 15:14:47,282 - BERTopic - Cluster - Completed ✓
2025-07-16 15:14:47,285 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-07-16 15:14:47,305 - BERTopic - Representation - Completed ✓


### View Topics

In [ ]:
topic_model_pos_tuned.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,69,-1_teman_perkuliahan_lingkungan_belajar,"[teman, perkuliahan, lingkungan, belajar, materi, dosen, kegiatan, menyenangkan, kuliah, pertemanan]","[lingkungan teman teman dosen membantu pembelajaran, perkuliahan lingkungan kampus nyaman kondusif belajar teman teman suportif aktif kegiatan pengalaman kuliah menyenangkan, teman teman mendukung membantu perkuliahan teman teman belajar bareng memahami materi]"
1,0,32,0_mahasiswa_pengalaman_mengikuti_mengasah,"[mahasiswa, pengalaman, mengikuti, mengasah, kerja, magang, kegiatan, kampus, disediakan, kemampuan]","[pembelajaran fleksibel dosen dosen mendukung kegiatan mahasiswa pengalaman organisasi mengasah keterampilan, program magang dimiliki binus university mengharuskan mahasiswa i melaksanakan magang mahasiswa i unggul hal pengalaman kerja dibandingkan fresh graduate universitas, program disediakan mengharuskan mahasiswa mengikuti kegiatan magang kegiatan pengalaman berguna memasuki dunia kerja]"
2,1,32,1_mata_materi_kuliah_data,"[mata, materi, kuliah, data, dosen, scientist, jurusan, kelas, bagus, fleksibel]","[mata kuliah diajarkan dosen pertemanan terbentuk kuliah, program magang materi kuliah mendukung bidang data science, mata kuliah kurikulum jurusan mata kuliah bagus bagus materi]"
3,2,20,2_fasilitas_kampus_lengkap_memadai,"[fasilitas, kampus, lengkap, memadai, belajar, akses, ruangan, tugas, disediakan, lab]","[fasilitas perpustakaan kampus lengkap nyaman belajar mandiri kelompok koleksi buku jurnal mendukung pengerjaan tugas skripsi ditambah suasana tenang kondusif fokus, lingkungan kampus mendukung gaya hidup sehat tersedia air minum ulang gratis fasilitas pejalan kaki memadai akses transportasi mudah pulang pergi menghawatirkan akses transportasi internal dosen departemen terbuka aspirasi kritik saran dikabulkan, fasilitas disediakan kampus lengkap mendukung proses belajar perpustakaan nyaman ruang diskusi bersantai fasilitas belajar efektif tugas tugas perkuliahan maksimal]"
4,3,19,3_dosen_mudah_mengajar_materi,"[dosen, mudah, mengajar, materi, dipahami, sistem, perkuliahan, padat, mengerti, murid]","[dosen kampus dosen mengajar materi dosen mudah diajak komunikasi, dosen dosen kampus materi terstruktur mengerti materi mudah, dosen dosen kampus materi terstruktur teman teman mengerti materi mudah]"
5,4,17,4_pelajaran_artikel_orang_materi,"[pelajaran, artikel, orang, materi, supportif, teman, praktikum, jadwal, belajar, dosen]","[pelajaran pelajaran diajarkan jurusan dosen dosen kompeten memahami materi enjoy belajar pertemanan salah hal senang pergi kampus capai bertemu teman kampus semangat belajar, pelajaran pelajaran kurikulum topik topik dibahas relevan jurusan dosen dosen pelajaran modern membantu pertemanan kampus bersifat toxic lingkungan kampus nyaman, menulis artikel paper tugas artikel menyalurkan kemampuan menulis menambah pengetahuan membaca artikel orang orang dijadikan dasar penulisan artikel]"
6,5,16,5_teman_senang_latar_konflik,"[teman, senang, latar, konflik, organisasi, kakak, orang, memiliki, koneksi, pikiran]","[senang lingkungan pertemanan perkuliahan mengenal orang latar mengapresiasi kinerja dosen mengajar, teman teman seru koneksi bidang teman teman bertukar pikiran membantu tugas pehamaman, senang berteman teman memiliki latar berbeda beda mengenal mengerti hal]"
7,6,15,6_bareng_kawan_teman_seru,"[bareng, kawan, teman, seru, kampus, suasana, asik, temen, berinteraksi, muda]","[lingkungan belajar teman teman belajar bareng teman ujian, teman teman lingkungan belajar bareng makan bareng kampus, teman teman bareng belajar bareng main bareng senang sedih bareng perkuliahan asik]"
8,7,11,7_hal_bertemu_orang_beda,"[hal, bertemu, orang, beda, berkuliah, proses, dapatkan, ilmu, belajar, emosi]","[proses belajar meregulasi emosi time management bertemu orang berbeda beda belajar hal buruk, lingkungan kampus belajar hal bertemu teman bertemu, bertemu orang orang latar hal bertemu orang orang heb

In [ ]:
topic_model_neg_tuned.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,87,-1_wifi_kampus_tugas_hal,"[wifi, kampus, tugas, hal, kelas, fasilitas, parkir, toilet, dosen, mahasiswa]","[perkuliahan hal i jadwal perkuliahan padat tugas tugas menumpuk bersamaan hal sulit membagi optimal mata kuliah penyampaian materi menarik aplikatif proses belajar menyenangkan berusaha mengambil pelajaran pengalaman, fasilitas kantin area parkir memadai dibandingkan mahasiswa jam makan siang kantin penuh sesak mencari parkir seringkali sulit, mengajar dosen membaca slides tersedia akses parkir memadai kampus mahasiswa mencari alternatif parkir hal hal mengganggu proses belajar diatasi]"
1,0,32,0_macet_ga_kampus_parkiran,"[macet, ga, kampus, parkiran, jalanan, kelas, masuk, hujan, besar, sempit]","[lingkungan kampus macet jalanan nyaman pejalan kaki seringkali telat masuk kelas, kampus macet parkira sempit telat masuk kelas, parkiran kampus anggrek macet jalanan susah parkir]"
2,1,26,1_materi_dosen_kelompok_kerja,"[materi, dosen, kelompok, kerja, niat, pembelajaran, pengajaran, nilai, belajar, penjelasan]","[hal i kuliah dosen dosen membawakan materi interaksi terbatas mahasiswa interaksi terbatas mahasiswa cepat bosan memperhatikan materi pembelajaran, kualitas dosen konsisten dosen memahami materi hal salah dosen membuka gpt sembunyi kelas materi pengajaran, dosen materi materi materi dosen materi hal menghambat belajar berpengaruh hasil ujian]"
3,2,25,2_toilet_fasilitas_kantin_sabun,"[toilet, fasilitas, kantin, sabun, tangan, tisu, kotor, bersih, cuci, kursi]","[toilet kotor sabun cuci tangan lingkungan kampus makan, fasilitas toilet disediakan tisu toilet bersih kantin kampus antrian beli makan, fasilitas kantin direnovasi sabun cuci tangan toilet]"
4,3,21,3_kuliah_jam_jadwal_mata,"[kuliah, jam, jadwal, mata, praktikum, pagi, padat, jurusan, magang, matkul]","[mata kuliah bersifat general kampus mempelajari mendaftar kampus belajar jam jam mata kuliah, jam mata kuliah gap mata kuliah mata kuliah bolak kampus, jadwal mata kuliah pagi jam kuliah pagi jam pagi masuk kelas jam]"
5,4,19,4_wifi_internet_terkoneksi_susah,"[wifi, internet, terkoneksi, susah, mengakses, absensi, kampus, lambat, konek, kuota]","[senang fasilitas wifi kampus sinyal mengakses internet, wifi kampus susah diakses kuota mengakses internet, wifi sulit terkoneksi susah internet stabil]"
6,5,17,5_mengajar_dosen_mengerti_mengajarkan,"[mengajar, dosen, mengerti, mengajarkan, materi, bersemangat, dimengerti, teori, mahasiswa, kompeten]","[dosen memilih mengajar duduk datar seharus dosen bersemangat mengajar mahasiswa mengerti bersemangat kelas, dosen dosen memahami materi mengajar sulit mengajarkan mahasiswa, dosen mengajar mahasiswa mengerti]"
7,6,16,6_orang_hal_sistem_kampus,"[orang, hal, sistem, kampus, mahasiswa, ujian, pressure, administrasi, fasilitas, website]","[cocok lingkungan sosial kampus slow living orang orang terkesan pemikiran kedaerahan kampus konservatif ngurusin orang nyaman berinteraksi teman teman perkuliahan isu utama isu dosen kompeten mengajar moody an mengajar mempengaruhi suasana kelas nilai mahir menguasai metode penelitian dituntut mendampingi mahasiswa metode penelitian ' a ' dikuasai hasil maksimal, orang orang kampus petra tenaga pendidik tu hingga rektor berisikan orang orang kompeten mempersulit malas mencerminkan nilai kristiani kampus kristen visi misi kristiani hal sebal fakultas arsitek anak tirikan kampus fakultas arsitek sipil nomer surabaya jurusan kedokteran bisnis dkv ilkom fasilitas terbaru terlengkap, berbicara hal dii sebenar dibenci fasilitas kampus ruang kelas toilet internet library pojok bersantai mendukung kehidupan perkuliahan perfect bayangkan kayak diluar negeri kampus kurikulum materi belajar gak relevan ppt singkat informatif dipakai belajar persiapan ujian kunci inti ilmu dosen ketemu dosen baca ppt menyuruh murid materi ppt pertemuan gak efektif untung sesi kayak responsi pengalaman terburuk skripsi dospem undi otomatis by system jabatan 

In [ ]:
topic_model_pos_tuned.visualize_barchart().show()

In [ ]:
topic_model_neg_tuned.visualize_barchart().show()

In [ ]:
topic_model_pos_tuned.visualize_documents(texts_pos)

In [ ]:
topic_model_neg_tuned.visualize_documents(texts_neg)

# **TES remove stopwords using Sastrawi**

In [ ]:
pip install -q Sastrawi

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic

stopwords = StopWordRemoverFactory().get_stop_words()
custom_stopwords = ['menyukai', 'suka', 'nya', 'saya', 'yang', 'dan', 'dengan', 'kurang']  # Added common words
stopwords.extend(custom_stopwords) # Use extend to add elements from custom_stopwords

vectorizer_model = CountVectorizer(stop_words=stopwords) # Use the combined stopwords list

embedding_model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
# embedding_model = SentenceTransformer("indobenchmark/indobert-base-p1")

topic_model_pos = BERTopic(embedding_model=embedding_model, vectorizer_model=vectorizer_model)
topic_model_neg = BERTopic(embedding_model=embedding_model, vectorizer_model=vectorizer_model)
topics_pos, probs_pos = topic_model_pos.fit_transform(texts_pos)
topics_neg, probs_neg = topic_model_neg.fit_transform(texts_neg)

In [ ]:
topic_model_pos.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,23,-1_dosen_teman_materi_kelas,"[dosen, teman, materi, kelas, baik, mahasiswa, artikel, lebih, fasilitas, diberikan]","[materi yang diajarkan teman teman yang saling mendukung satu sama lain dalam proses study dosen yang mampu mengajar dengan baik fasilitas yang ada lokasi yang cukup strategis, saya menyukai aktivitas belajar bersama dengan teman teman saya karena saya lebih bisa mengerti materi yang akan diujiankan dibandingkan hanya dengan dari materi yang diberikan dosen saja, saya menyukai sistem dosen dosen fasilitas di mampus saya karena dapat mengakomodasi pembelajaran saya sehari hari dengan baik dan terstruktur walaupun terkadang sistem nya bermasalah namun sistem yang ada sudah cukup membantu dengan ada nya ketepatan waktu dan time efficiency yang diberikan selain itu dosen dosen kami memiliki kemampuan yang kompeten dan memiliki pengalaman yang relevan dan juga sikap yang baik dan kooperatif fasilitas kampus yang diberikan juga baik semua mendukung seperti toilet ekskalator lift wifi dan lain nya yang cukup untuk memberikan pengalaman yang baik]"
1,0,250,0_teman_kampus_materi_dosen,"[teman, kampus, materi, dosen, sangat, belajar, lingkungan, perkuliahan, baik, kuliah]","[saya menyukai dosen dosen di kampus saya karena bisa menjelaskan dengan baik dan materi yang diberikan juga sangat terstruktur oleh karena itu saya dan teman teman saya bisa mengerti materi nya dengan mudah, dosen dosen kampus sangat friendly dan juga menjelaskan materi dengan baik sehingga mudah dipelajari oleh karena itu saya dan teman teman saya sangat menyukai pelajaran selama masa perkuliahan, selama masa perkuliahan saya sangat menyukai lingkungan kampus yang nyaman dan kondusif untuk belajar selain itu teman teman yang suportif dan aktif dalam berbagai kegiatan juga membuat pengalaman kuliah saya jadi lebih menyenangkan]"
2,1,28,1_organisasi_pengalaman_banyak_mendapatkan,"[organisasi, pengalaman, banyak, mendapatkan, diskusi, teman, dosen, belajar, kegiatan, lebih]","[saya menyukai pengalaman organisasi yang saya ikuti selama perkuliahan saya karena mengikuti organisasi dan event event saya mendapatkan banyak pengalaman baru dan teman teman baru, selama masa perkuliahan saya sangat menyukai kesempatan untuk belajar hal hal baru yang relevan dengan minat saya saya juga menikmati diskusi diskusi di kelas yang membuka wawasan serta suasana akademik yang mendorong saya untuk berpikir kritis dan berkembang secara pribadi selain itu saya juga senang bisa terlibat dalam berbagai kegiatan organisasi dan proyek kolaboratif yang memperluas jaringan dan keterampilan saya di luar kelas, selama masa perkuliahan saya sangat menikmati suasana diskusi di dalam kelas yang mendorong saya untuk berpikir kritis dan terbuka terhadap berbagai sudut pandang saya juga sangat menyukai kesempatan untuk terlibat dalam organisasi kemahasiswaan karena melalui kegiatan tersebut saya dapat mengembangkan keterampilan kepemimpinan bekerja dalam tim serta memperluas jaringan pertemanan lintas jurusan dan angkatan pengalaman pengalaman ini tidak hanya memperkaya wawasan akademik saya tetapi juga membentuk karakter dan kesiapan saya untuk menghadapi dunia profesional]"


In [ ]:
topic_model_neg.get_topic_info()[['Topic', 'Count']]

,Topic,Count
0,-1,6
1,0,190
2,1,51
3,2,33
4,3,21


In [ ]:
topic_model_neg.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,6,-1_tisu_fasilitas_wifi_term,"[tisu, fasilitas, wifi, term, pertemanan, diperbaiki, parkir, toilet, tersedia, sesuai]","[saya tidak suka dengan sistem absen nya yang menurut saya cukup strict selain itu juga fasilitas nya yang sangat kurang memadai seperti contoh wifi yang buruk parkir yang sempit dan toilet yang gaada tisu selain itu saya juga kurang setuju dengan aturan seperti beelingua sat dan conserve hour yang menurut saya beban dan tidak berguna, saya tidak menyukai koneksi wifi di kampus karena sering kali sangat lambat sehingga menghambat saya dalam mengakses materi kuliah dan mengerjakan tugas secara online selain itu fasilitas toilet di kampus juga kurang memadai karena sering tidak tersedia tisu lift yang tersedia pun hanya empat unit untuk melayani sebelas lantai sehingga sering terjadi antrean panjang dan memakan waktu lama untuk berpindah lantai, saya tidak menyukai fasilitas di kuliah seperti tidak ada toilet tisu eskalator mati dan wifi yang tidak bagus untuk dipakai di kampus terutama di perpus pergaulan dari pertemanan bisa menjadi tidak disukai dikarenakan orang tersebut tidak sesuai dengan prinsip kita ataupun terdapat tindakan yang tidak sesuai dengan normalisasi kita bagus nya menjadi filterasi mana yang long term dan short term di dunia pertemanan]"
1,0,190,0_dosen_kampus_mahasiswa_waktu,"[dosen, kampus, mahasiswa, waktu, beberapa, kelas, kuliah, materi, fasilitas, banyak]","[berbicara soal hal yang tidak disukai sebenar nya tidak ada yang betul betul dibenci fasilitas kampus baik ruang kelas toilet internet library ataupun pojok bersantai sudah cukup mendukung kehidupan perkuliahan walaupun ya tidak semua nya perfect ya seperti yang kita bayangkan kayak diluar luar negeri atau kampus lain nah soal kurikulum atau materi belajar menurut saya terkadang ada beberapa yang gak relevan dengan keadaan saat ini ppt kadang terlalu singkat dan tidak semua nya informatif apalagi kalau saat dipakai buat belajar persiapan ujian alhasil ya memang kunci atau inti ilmu itu masih dari dosen itu sendiri tetapi misal ketemu dosen yang hanya baca ppt atau menyuruh murid yang menjelaskan materi ppt tiap pertemuan menurut saya itu gak efektif sangat sih untung ada sesi kayak responsi dan lain sebagai nya pengalaman terburuk saya ya waktu skripsi ya secara mendapat dospem yang di undi atau otomatis by system meski jabatan nya tinggi namun susah dihubungi dan kurang membimbing sih oh ya tambahan lagi di binus ini susah sangat kalau mau membuat ijin atau susulan dan lain sebagai nya jadi ya kebanyakan temen saya memilih untuk pakai jatah absen atau terpakai ikut semester pendek walaupun ya saya paham ini tentang disiplin oiya soal kelas pengganti kadang sudah sampai kampus malah dosen bukan nya mengabari tidak bisa hadir malahan ssc yang beri tahu di detik detik akhir parkiran dan akses jalan binus kemanggisan jujur sempit sih apalagi ada pelajaran yang perlu pindah kampus kanapa ga bangun gedung per fakultas aja gtu ya dalam benak saya delain itu kurang ruang terbuka atau outdoor untuk bersantai ya walaupun emang ga mungkin ada taman dan danau kayak di univ lain gtu misal nya wkwkw, selama masa perkuliahan hal yang kurang saya sukai adalah ketika jadwal perkuliahan terlalu padat dan tugas tugas menumpuk dalam waktu yang bersamaan hal ini kadang membuat sulit untuk membagi waktu secara optimal selain itu ada beberapa mata kuliah yang penyampaian materi nya kurang menarik atau kurang aplikatif sehingga membuat proses belajar menjadi kurang menyenangkan namun saya tetap berusaha mengambil pelajaran dari setiap pengalaman tersebut, saya tidak menyukai jadwal kuliah yang sering berubah secara mendadak akibat dosen yang tidak hadir karena membuat saya kesulitan dalam mengatur waktu untuk kegiatan lain di luar perkuliahan selain itu beberapa kelas terasa terlalu padat dalam satu hari sehingga cukup melelahkan dan mengurangi fokus saat mengikuti mata kuliah terakhir]"
2,1,51,1

In [ ]:
topic_model_pos.visualize_barchart().show()

In [ ]:
topic_model_neg.visualize_barchart().show()

# **Evaluation**

**Dokumentasi tuning**: https://docs.google.com/spreadsheets/d/1PG2Y5-kA4ENGeOc-iRbEOYQKpEy0A7wBGyYU8PS_dOs/edit?gid=529725999#gid=529725999

run this but don't restart the session

In [11]:
!pip install -q --upgrade --force-reinstall numpy gensim

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nlp-id 0.1.19.0 requires huggingface-hub==0.31.1, but you have huggingface-hub 0.34.3 which is incompatible.
nlp-id 0.1.19.0 requires numpy==2.0.2, but you have numpy 1.26.4 which is incompatible.
nlp-id 0.1.19.0 requires scipy==1.15.3, but you have scipy 1.13.1 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "

In [17]:
pip install -q rbo

In [ ]:
from itertools import combinations
# from rbo import rbo
# import rbo

from gensim import corpora
from gensim.models.coherencemodel import CoherenceModel

In [ ]:
def get_all_topics(topic_model_pos, topic_model_neg):
    topics_pos = topic_model_pos.get_topics()
    topics_neg = topic_model_neg.get_topics()

    all_topics_pos = [[word for word, _ in word_list] for word_list in topics_pos.values()]
    all_topics_neg = [[word for word, _ in word_list] for word_list in topics_neg.values()]

    print("="*5,"POSITIVE", "="*5)
    print(len(all_topics_pos))
    print(all_topics_pos)
    print("\n", "="*5,"NEGATIVE", "="*5)
    print(len(all_topics_neg))
    print(all_topics_neg)

    return all_topics_pos, all_topics_neg

all_topics_pos, all_topics_neg = get_all_topics(topic_model_pos, topic_model_neg)

NameError: name 'topic_model_pos' is not defined

## 1. Topic Coherence

https://radimrehurek.com/gensim/models/coherencemodel.html

In [19]:
pip install -q --upgrade gensim

In [12]:
from gensim.utils import simple_preprocess

def calculate_coherence_score(texts, all_topics, coherence_scores):
  tokenized_docs = [simple_preprocess(doc) for doc in texts]

  # Build dictionary and corpus
  dictionary = corpora.Dictionary(tokenized_docs)
  corpus = [dictionary.doc2bow(text) for text in tokenized_docs]

  scores = {}

  # Compute all coherence types
  for coherence in coherence_scores:
      model = CoherenceModel(
          topics=all_topics,
          texts=tokenized_docs,
          corpus=corpus,
          dictionary=dictionary,
          coherence=coherence
      )
      scores[coherence] = round(model.get_coherence(), 3)

  for metric, score in scores.items():
    print(f"{metric}: {score}")
  return scores

In [13]:
coherence_scores = ['c_v', 'u_mass', 'c_uci', 'c_npmi']

print("Topic Coherence Score for Positive Sentiment")
positive_coherence_scores = calculate_coherence_score(texts_pos, all_topics_pos, coherence_scores)
print("Topic Coherence Score for Negative Sentiment")
negative_coherence_scores = calculate_coherence_score(texts_neg, all_topics_neg, coherence_scores)

Topic Coherence Score for Positive Sentiment


NameError: name 'all_topics_pos' is not defined

## 2. Topic Diversity

https://github.com/silviatti/topic-model-diversity/blob/master/diversity_metrics.py

but cannot use rbo pls help solve

In [14]:
import rbo
import itertools
import numpy as np

def calculate_irbo(all_topics, topk=10):
    """
    all_topics: list of topic words (each topic is a list of top-k words)
    topk: how many top words to use per topic
    """
    T = len(all_topics)
    n = T * (T - 1) / 2
    rbo_sum = 0

    for i in range(1, T):
        for j in range(i):
            l1 = all_topics[i][:topk]
            l2 = all_topics[j][:topk]
            rbo_score = rbo.RankingSimilarity(l1, l2).rbo()
            rbo_sum += rbo_score

    irbo_score = 1 - (rbo_sum / n)
    return round(irbo_score, 3)


In [15]:
from gensim.models.coherencemodel import CoherenceModel
from gensim import corpora
from gensim.utils import simple_preprocess

def evaluate_topics(texts, topic_model, coherence_types=['c_v', 'u_mass', 'c_uci', 'c_npmi'], topk=10):
    # Tokenized text for coherence
    tokenized_docs = [simple_preprocess(doc) for doc in texts]
    dictionary = corpora.Dictionary(tokenized_docs)
    corpus = [dictionary.doc2bow(text) for text in tokenized_docs]

    # Get all topic words
    all_topics = [ [word for word, _ in topic_model.get_topic(topic_id)[:topk]]
                   for topic_id in topic_model.get_topics().keys()
                   if topic_id != -1 and topic_model.get_topic(topic_id) is not None ]

    # Topic Coherence
    coherence_scores = {}
    for ctype in coherence_types:
        cm = CoherenceModel(topics=all_topics, texts=tokenized_docs, corpus=corpus, dictionary=dictionary, coherence=ctype)
        coherence_scores[ctype] = round(cm.get_coherence(), 3)

    # Topic Diversity (IRBO)
    irbo_score = calculate_irbo(all_topics, topk=topk)

    # Print all
    for ctype, score in coherence_scores.items():
        print(f"Coherence ({ctype}): {score}")
    print(f"IRBO Topic Diversity: {irbo_score}")

    return coherence_scores, irbo_score


In [16]:
embedding_models = [
    "paraphrase-multilingual-MiniLM-L12-v2",
    "indobenchmark/indobert-base-p1"
]

umap_params = [
    {"n_neighbors": 7, "min_dist": 0.0, "metric": "cosine"},
    {"n_neighbors": 5, "min_dist": 0.1, "metric": "euclidean"},
    {"n_neighbors": 5, "min_dist": 0.0, "metric": "cosine"},
    {"n_neighbors": 7, "min_dist": 0.1, "metric": "euclidean"},
    {"n_neighbors": 10, "min_dist": 0.1, "metric": "cosine"},
    {"n_neighbors": 15, "min_dist": 0.3, "metric": "euclidean"}
]

hdbscan_params = [
    {"min_cluster_size": 10, "min_samples": 3, "cluster_selection_epsilon": 0.0},
    {"min_cluster_size": 5, "min_samples": 3, "cluster_selection_epsilon": 0.5},
    {"min_cluster_size": 5, "min_samples": 3, "cluster_selection_epsilon": 0.3},
    {"min_cluster_size": 10, "min_samples": 3, "cluster_selection_epsilon": 0.3},
    {"min_cluster_size": 15, "min_samples": 5, "cluster_selection_epsilon": 0.0},
    {"min_cluster_size": 10, "min_samples": 5, "cluster_selection_epsilon": 0.5},
]

bertopic_params = [
    {"top_n_words": 5, "nr_topics": 3},
    {"top_n_words": 5, "nr_topics": 4},
    {"top_n_words": 5, "nr_topics": 5},
    {"top_n_words": 5, "nr_topics": 6},
    {"top_n_words": 10, "nr_topics": 3},
    {"top_n_words": 10, "nr_topics": 4},
    {"top_n_words": 10, "nr_topics": 5},
    {"top_n_words": 10, "nr_topics": 6},
]

In [17]:
def run_topic_modeling(label):
    texts = texts_pos if label == "positive" else texts_neg
    results = []

    for emb in embedding_models:
        embedding_model = SentenceTransformer(emb)

        for u in umap_params:
            umap_model = None if u is None else UMAP(**u)

            for h in hdbscan_params:
                hdbscan_model = None if h is None else hdbscan.HDBSCAN(**h)

                for b in bertopic_params:
                    topic_model = BERTopic(
                        embedding_model=embedding_model,
                        umap_model=umap_model,
                        hdbscan_model=hdbscan_model,
                        top_n_words=b["top_n_words"],
                        nr_topics=b["nr_topics"],
                        verbose=False
                    )

                    topics, probs = topic_model.fit_transform(texts)

                    # Evaluate topics
                    coherence_scores, irbo_score = evaluate_topics(texts, topic_model)

                    # Filter topics
                    topics_dict = topic_model.get_topics()
                    topics_filtered = {k: v for k, v in topics_dict.items() if k != -1 and v is not None}

                    # Create entry
                    results.append({
                        "Label": label,
                        "Embedding Model": emb,
                        "UMAP": u,
                        "HDBSCAN": h,
                        "BERTopic Params": b,
                        "Num Topics": len(topics_filtered),
                        "Topics List": [ [word for word, _ in words] for words in topics_filtered.values()],
                        **coherence_scores,
                        "IRBO": irbo_score
                    })

    return results

In [18]:
positive_results = run_topic_modeling("positive")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Coherence (c_v): 0.454
Coherence (u_mass): -6.088
Coherence (c_uci): -3.266
Coherence (c_npmi): -0.069
IRBO Topic Diversity: 0.453
Coherence (c_v): 0.528
Coherence (u_mass): -1.798
Coherence (c_uci): 0.219
Coherence (c_npmi): 0.061
IRBO Topic Diversity: 0.8
Coherence (c_v): 0.45
Coherence (u_mass): -6.101
Coherence (c_uci): -4.493
Coherence (c_npmi): -0.101
IRBO Topic Diversity: 0.951
Coherence (c_v): 0.639
Coherence (u_mass): -2.205
Coherence (c_uci): -0.025
Coherence (c_npmi): 0.089
IRBO Topic Diversity: 0.962
Coherence (c_v): 0.312
Coherence (u_mass): -3.295
Coherence (c_uci): -1.777
Coherence (c_npmi): -0.039
IRBO Topic Diversity: 0.733
Coherence (c_v): 0.403
Coherence (u_mass): -2.926
Coherence (c_uci): -1.143
Coherence (c_npmi): -0.01
IRBO Topic Diversity: 0.809
Coherence (c_v): 0.568
Coherence (u_mass): -3.902
Coherence (c_uci): -1.863
Coherence (c_npmi): 0.036
IRBO Topic Diversity: 0.905
Coherence (c_v): 0.384
Coherence (u_mass): -5.515
Coherence (c_uci): -3.04
Coherence (c_npm

In [19]:
# Convert to DataFrame
df_pos = pd.DataFrame(positive_results)

# If topics are lists, convert to string to make them readable in Excel
df_pos["Topics List"] = df_pos["Topics List"].apply(lambda x: str(x))

# Export to Excel
df_pos.to_excel("positive_topic_modeling_results.xlsx", index=False)

In [20]:
negative_results = run_topic_modeling("negative")

Coherence (c_v): 0.58
Coherence (u_mass): -1.52
Coherence (c_uci): 0.303
Coherence (c_npmi): 0.102
IRBO Topic Diversity: 0.843
Coherence (c_v): 0.594
Coherence (u_mass): -1.579
Coherence (c_uci): 0.323
Coherence (c_npmi): 0.115
IRBO Topic Diversity: 0.904
Coherence (c_v): 0.569
Coherence (u_mass): -3.956
Coherence (c_uci): -2.688
Coherence (c_npmi): -0.007
IRBO Topic Diversity: 0.868
Coherence (c_v): 0.548
Coherence (u_mass): -3.45
Coherence (c_uci): -1.507
Coherence (c_npmi): 0.037
IRBO Topic Diversity: 0.939
Coherence (c_v): 0.476
Coherence (u_mass): -2.485
Coherence (c_uci): -0.539
Coherence (c_npmi): 0.039
IRBO Topic Diversity: 0.847
Coherence (c_v): 0.391
Coherence (u_mass): -6.111
Coherence (c_uci): -3.616
Coherence (c_npmi): -0.081
IRBO Topic Diversity: 0.858
Coherence (c_v): 0.421
Coherence (u_mass): -3.606
Coherence (c_uci): -1.405
Coherence (c_npmi): 0.031
IRBO Topic Diversity: 0.878
Coherence (c_v): 0.46
Coherence (u_mass): -3.834
Coherence (c_uci): -1.157
Coherence (c_npmi)

Coherence (c_v): 0.591
Coherence (u_mass): -1.683
Coherence (c_uci): 0.124
Coherence (c_npmi): 0.078
IRBO Topic Diversity: 1.0
Coherence (c_v): 0.453
Coherence (u_mass): -6.938
Coherence (c_uci): -4.277
Coherence (c_npmi): -0.071
IRBO Topic Diversity: 1.0
Coherence (c_v): 0.651
Coherence (u_mass): -1.536
Coherence (c_uci): 0.519
Coherence (c_npmi): 0.148
IRBO Topic Diversity: 0.924
Coherence (c_v): 0.636
Coherence (u_mass): -2.107
Coherence (c_uci): -0.329
Coherence (c_npmi): 0.099
IRBO Topic Diversity: 0.89
Coherence (c_v): 0.42
Coherence (u_mass): -2.671
Coherence (c_uci): -0.997
Coherence (c_npmi): -0.009
IRBO Topic Diversity: 0.882
Coherence (c_v): 0.416
Coherence (u_mass): -4.512
Coherence (c_uci): -3.726
Coherence (c_npmi): -0.112
IRBO Topic Diversity: 0.827
Coherence (c_v): 0.388
Coherence (u_mass): -5.327
Coherence (c_uci): -3.004
Coherence (c_npmi): -0.047
IRBO Topic Diversity: 0.882
Coherence (c_v): 0.308
Coherence (u_mass): -6.357
Coherence (c_uci): -3.25
Coherence (c_npmi):

In [21]:
# Convert to DataFrame
df_neg = pd.DataFrame(negative_results)

# If topics are lists, convert to string to make them readable in Excel
df_neg["Topics List"] = df_neg["Topics List"].apply(lambda x: str(x))

# Export to Excel
df_neg.to_excel("negative_topic_modeling_results.xlsx", index=False)

In [22]:
top5_pos = df_pos.sort_values(by="c_v", ascending=False).head(5)
top5_neg = df_neg.sort_values(by="c_v", ascending=False).head(5)

In [23]:
def visualize_top_models(top_models_df, label, texts):
    for idx, row in top_models_df.iterrows():
        print(f"\n==========================")
        print(f"Top Model {idx+1} ({label.upper()} Data)")
        print(f"==========================")

        # Load components
        emb_model_name = row["Embedding Model"]
        emb_model = SentenceTransformer(emb_model_name)
        umap_params = row["UMAP"]
        hdbscan_params = row["HDBSCAN"]
        bertopic_params = row["BERTopic Params"]
        cv_score = row["c_v"]

        # Print full parameter config
        print("Parameters:")
        print(f"Embedding Model  : {emb_model_name}")
        print(f"UMAP Params      : {umap_params}")
        print(f"HDBSCAN Params   : {hdbscan_params}")
        print(f"BERTopic Params  : {bertopic_params}")
        print(f"C_V Coherence Score: {cv_score:.4f}")

        # Build models
        umap_model = None if pd.isna(umap_params) else UMAP(**umap_params)
        hdbscan_model = None if pd.isna(hdbscan_params) else hdbscan.HDBSCAN(**hdbscan_params)

        # Rebuild and fit model
        topic_model = BERTopic(
            embedding_model=emb_model,
            umap_model=umap_model,
            hdbscan_model=hdbscan_model,
            top_n_words=bertopic_params["top_n_words"],
            nr_topics=bertopic_params["nr_topics"]
        )

        topics, probs = topic_model.fit_transform(texts)

        # Topic Info
        print("\nTopic Info:")
        display(topic_model.get_topic_info())

        # Visualizations
        print("\nBarchart:")
        topic_model.visualize_barchart(top_n_topics=6).show()

        print("\nDocument Visualization:")
        topic_model.visualize_documents(texts).show()

In [24]:
visualize_top_models(top5_pos, "positive", texts_pos)


Top Model 556 (POSITIVE Data)


Parameters:
Embedding Model  : indobenchmark/indobert-base-p1
UMAP Params      : {'n_neighbors': 15, 'min_dist': 0.3, 'metric': 'euclidean'}
HDBSCAN Params   : {'min_cluster_size': 10, 'min_samples': 3, 'cluster_selection_epsilon': 0.3}
BERTopic Params  : {'top_n_words': 5, 'nr_topics': 6}
C_V Coherence Score: 0.7050

Topic Info:


,Topic,Count,Name,Representation,Representative_Docs
0,-1,24,-1_kampus_teman_makanan_binus,"[kampus, teman, makanan, binus, materi]","[dosen kampus penjelasan mudah dipahami teman teman kampus, makanan kampus bakmi efata disekitar kampus makanan enak, daerah kampus makanan explore makanan jajan teman teman terdekat seru gak nyangka circle real sma]"
1,0,249,0_teman_dosen_materi_kampus,"[teman, dosen, materi, kampus, belajar]","[dosen dosen kampus materi terstruktur teman teman mengerti materi mudah, perkuliahan lingkungan kampus nyaman kondusif belajar teman teman suportif aktif kegiatan pengalaman kuliah menyenangkan, teman teman mendukung membantu perkuliahan teman teman belajar bareng memahami materi]"
2,1,28,1_fasilitas_kampus_lengkap_makanan,"[fasilitas, kampus, lengkap, makanan, memadai]","[fasilitas kampus bagus program kemahasiswaan mendukung mahasiswa belajar, fasilitas fasilitas kampus fasilitas memudahkan darurat menge print tugas proposal tersedia printing seberang kampus, fasilitas disediakan kampus lengkap mendukung proses belajar perpustakaan nyaman ruang diskusi bersantai fasilitas belajar efektif tugas tugas perkuliahan maksimal]"



Barchart:



Document Visualization:


...


Top Model 297 (POSITIVE Data)


Parameters:
Embedding Model  : indobenchmark/indobert-base-p1
UMAP Params      : {'n_neighbors': 7, 'min_dist': 0.0, 'metric': 'cosine'}
HDBSCAN Params   : {'min_cluster_size': 5, 'min_samples': 3, 'cluster_selection_epsilon': 0.5}
BERTopic Params  : {'top_n_words': 5, 'nr_topics': 3}
C_V Coherence Score: 0.6980

Topic Info:


,Topic,Count,Name,Representation,Representative_Docs
0,0,5,0_materi_sistem_binus_mengakses,"[materi, sistem, binus, mengakses, binusmaya]","[ketersediaan materi lengkap aplikasi binusmaya recording kelas mereview materi, sistem pembelajaran binus terstruktur teregulasi mencari materi pertemuan mengakses langsung binusmaya, materi materi dibilang terstruktur mudah memahami mata pelajaran jurusan senang sistem binus merekam kelas mengakses informasi tangkap kelas]"
1,1,18,1_diskusi_belajar_kelas_dosen,"[diskusi, belajar, kelas, dosen, pengalaman]","[perkuliahan menikmati suasana diskusi kelas mendorong berpikir kritis terbuka sudut pandang kesempatan terlibat organisasi kemahasiswaan kegiatan mengembangkan keterampilan kepemimpinan tim memperluas jaringan pertemanan lintas jurusan angkatan pengalaman pengalaman memperkaya wawasan akademik membentuk karakter kesiapan menghadapi dunia profesional, perkuliahan kesempatan belajar hal hal relevan minat menikmati diskusi diskusi kelas membuka wawasan suasana akademik mendorong berpikir kritis berkembang pribadi senang terlibat kegiatan organisasi proyek kolaboratif memperluas jaringan keterampilan kelas, lingkungan pertemanan temui perkuliahan supportive so far menjatuhkan merugikan opportunity disediakan kampus menambah penghasilan membuka lowongan pekerjaan part time full time mahasiswa matang dunia kerja lulus proses pembelajaran sebenar binus nyaman fasilitas infrastruktur materi kurikulum dipahami sesuai standar terbaru tren terbaru program program unit unit layanan konseling layanan riset membantu mahasiwa kebutuhan individu berbeda beda inti pengalaman pengalaman dapatkan kuliah bertemu circle pertemanan membangun solid fl fp melatih public speaking join ukm himpunan belajar project event management teamwork part time magang it division bina nusantara berperan membentuk binus menghasilkan lulusan berkualitas salah program event partnership penelitian ditawarkan murid aktif explore gtu ambis setuju setuju binus dibilang bagus]"
2,2,278,2_teman_kampus_dosen_materi,"[teman, kampus, dosen, materi, belajar]","[dosen dosen kampus materi terstruktur teman teman mengerti materi mudah, perkuliahan lingkungan kampus nyaman kondusif belajar teman teman suportif aktif kegiatan pengalaman kuliah menyenangkan, teman teman mendukung membantu perkuliahan teman teman belajar bareng memahami materi]"



Barchart:



Document Visualization:


In [25]:
visualize_top_models(top5_neg, "negative", texts_neg)


Top Model 212 (NEGATIVE Data)
Parameters:
Embedding Model  : paraphrase-multilingual-MiniLM-L12-v2
UMAP Params      : {'n_neighbors': 10, 'min_dist': 0.1, 'metric': 'cosine'}
HDBSCAN Params   : {'min_cluster_size': 5, 'min_samples': 3, 'cluster_selection_epsilon': 0.3}
BERTopic Params  : {'top_n_words': 5, 'nr_topics': 6}
C_V Coherence Score: 0.6900

Topic Info:


,Topic,Count,Name,Representation,Representative_Docs
0,-1,6,-1_nyaman_sistem_online_kendala,"[nyaman, sistem, online, kendala, praktikum]","[kekurangan tisu fasilitas toilet kursi kayu disediakan nyaman bilang schedule fleksibel kelas malam enak diikuti, sistem edunex kampus lambat error kuis ujian fasilitas toilet gedung sempit bergerak fakta gedung kuliah kampus memiliki lift, nyaman fasilitas toilet minim air nyaman sistem pembelajaran mata kuliah menurunkan efektivitas pembelajaran jadwal perkuliahan berubah sistem penjadwalan kuliah disesuaikan dosen]"
1,0,181,0_dosen_mahasiswa_kampus_kuliah,"[dosen, mahasiswa, kampus, kuliah, materi]","[berbicara hal dii sebenar dibenci fasilitas kampus ruang kelas toilet internet library pojok bersantai mendukung kehidupan perkuliahan perfect bayangkan kayak diluar negeri kampus kurikulum materi belajar gak relevan ppt singkat informatif dipakai belajar persiapan ujian kunci inti ilmu dosen ketemu dosen baca ppt menyuruh murid materi ppt pertemuan gak efektif untung sesi kayak responsi pengalaman terburuk skripsi dospem undi otomatis by system jabatan susah dihubungi membimbing oh tambahan binus susah ijin susulan kebanyakan temen memilih pakai jatah absen terpakai semester pendek paham disiplin oiya kelas pengganti kampus dosen mengabari hadir ssc detik detik parkiran akses jalan binus kemanggisan jujur sempit pelajaran pindah kampus kanapa ga bangun gedung fakultas gtu benak delain ruang terbuka outdoor bersantai ga taman danau kayak univ gtu wkwkw, dosen dosen memahami materi mengajar sulit mengajarkan mahasiswa, perkuliahan hal i jadwal perkuliahan padat tugas tugas menumpuk bersamaan hal sulit membagi optimal mata kuliah penyampaian materi menarik aplikatif proses belajar menyenangkan berusaha mengambil pelajaran pengalaman]"
2,1,48,1_wifi_kampus_koneksi_susah,"[wifi, kampus, koneksi, susah, absensi]","[fasilitas kampus wifi terputus koneksi jaringan lancar, koneksi jaringan wifi kampus wifi terkonek aktivitas kampus kesulitan termaksa memakai data hotspot teman aktivitas krusial absensi kelas wifi, wifi disediakan kampus koneksi jaringan kesulitan tugas absen]"
3,2,30,2_toilet_wifi_tisu_fasilitas,"[toilet, wifi, tisu, fasilitas, bersih]","[fasilitas toilet bersih tisu mencukupi wifi terkoneksi tugas lancar kampus, fasilitas toilet bersih tisu mencukupi wifi terkoneksi tugas lancar kampus, fasilitas toilet bersih tisu mencukupi wifi terkoneksi tugas lancar kampus]"
4,3,29,3_toilet_tisu_fasilitas_tissue,"[toilet, tisu, fasilitas, tissue, bersih]","[dosen ramah toilet lantai kotor toilet gedung bersih toilet gedung penuh, fasilitas kuliah toilet parkir toilet bersih kekurangan tisu parkir teratur penuh, fasilitas toilet disediakan tisu toilet bersih kantin kampus antrian beli makan]"
5,4,7,4_lift_penuh_fasilitas_sistem,"[lift, penuh, fasilitas, sistem, eskalator]","[fasilitas parkir binus alam sutera penuh fasilitas lift macet orang, luas kapasitas kampus tempati kegiatan kelas tercukupi hal hal lift kantin memiliki kapasitas sepadan serapan mahasiswa berasa kampus kegiatan kampus terhambat lambat, fasilitas kampus efektif lift penuh menyebabkan tangga darurat melelahkan sistem digital kampus maksimal bug sistem pembayaran sks transfer sistem terbaca komplain student service]"



Barchart:



Document Visualization:



Top Model 483 (NEGATIVE Data)


...


Top Model 484 (NEGATIVE Data)


Parameters:
Embedding Model  : indobenchmark/indobert-base-p1
UMAP Params      : {'n_neighbors': 10, 'min_dist': 0.1, 'metric': 'cosine'}
HDBSCAN Params   : {'min_cluster_size': 10, 'min_samples': 3, 'cluster_selection_epsilon': 0.0}
BERTopic Params  : {'top_n_words': 5, 'nr_topics': 6}
C_V Coherence Score: 0.6600

Topic Info:


,Topic,Count,Name,Representation,Representative_Docs
0,-1,38,-1_kampus_wifi_mahasiswa_toilet,"[kampus, wifi, mahasiswa, toilet, dosen]","[fasilitas wifi kelas koneksi wifi terputus hal kesulitan materi link digital, fasilitas kampus wifi terputus koneksi jaringan lancar, koneksi jaringan wifi kampus wifi terkonek aktivitas kampus kesulitan termaksa memakai data hotspot teman aktivitas krusial absensi kelas wifi]"
1,0,108,0_dosen_materi_kuliah_mata,"[dosen, materi, kuliah, mata, tugas]","[dosen materi materi materi dosen materi hal menghambat belajar berpengaruh hasil ujian, dosen materi menarik dosen penjelasan tugas, dosen dosen memahami materi mengajar sulit mengajarkan mahasiswa]"
2,1,72,1_toilet_fasilitas_kampus_tisu,"[toilet, fasilitas, kampus, tisu, wifi]","[fasilitas toilet bersih tisu mencukupi wifi terkoneksi tugas lancar kampus, fasilitas toilet bersih tisu mencukupi wifi terkoneksi tugas lancar kampus, fasilitas toilet bersih tisu mencukupi wifi terkoneksi tugas lancar kampus]"
3,2,35,2_kampus_mahasiswa_hal_sistem,"[kampus, mahasiswa, hal, sistem, orang]","[kekakuan sistem kampus fleksibel mahasiswa absen pengumpulan tugas sepenuh diatur sistem menoleransi keterlambatan keterlambatan mahasiswa mengumpulkan tugas disebabkan hal kendali mahasiswa maintenance website kampus pemberitahuan, sistem unversitas cenderung kacau internal kampus memiliki ketidakjelasan proses penyampaian informasi lambat hal hal terpancarkan pemberitahuan informasi mahasiswa terlambat perubahan jadwal peraturan hal hal terkesan tacid knowledge mahasiswa kampus universitas meningkatkan operasional, berbicara hal dii sebenar dibenci fasilitas kampus ruang kelas toilet internet library pojok bersantai mendukung kehidupan perkuliahan perfect bayangkan kayak diluar negeri kampus kurikulum materi belajar gak relevan ppt singkat informatif dipakai belajar persiapan ujian kunci inti ilmu dosen ketemu dosen baca ppt menyuruh murid materi ppt pertemuan gak efektif untung sesi kayak responsi pengalaman terburuk skripsi dospem undi otomatis by system jabatan susah dihubungi membimbing oh tambahan binus susah ijin susulan kebanyakan temen memilih pakai jatah absen terpakai semester pendek paham disiplin oiya kelas pengganti kampus dosen mengabari hadir ssc detik detik parkiran akses jalan binus kemanggisan jujur sempit pelajaran pindah kampus kanapa ga bangun gedung fakultas gtu benak delain ruang terbuka outdoor bersantai ga taman danau kayak univ gtu wkwkw]"
4,3,28,3_macet_kampus_parkiran_ga,"[macet, kampus, parkiran, ga, kelas]","[lingkungan kampus macet jalanan nyaman pejalan kaki seringkali telat masuk kelas, kampus macet parkira sempit telat masuk kelas, parkiran kampus anggrek macet jalanan susah parkir]"
5,4,20,4_wifi_internet_susah_terkoneksi,"[wifi, internet, susah, terkoneksi, kampus]","[wifi kampus terhubung hal internet susah, wifi kampus susah diakses kuota mengakses internet, wifi sulit terkoneksi susah internet stabil]"



Barchart:



Document Visualization:


### Brandon OLD VERSION

In [ ]:
# from sentence_transformers import SentenceTransformer
# from bertopic import BERTopic
# from umap import UMAP
# import hdbscan

# for emb in embedding_models:
#     embedding_model = SentenceTransformer(emb)

#     for u in umap_params:
#         umap_model = UMAP(n_neighbors=u["n_neighbors"], min_dist=u["min_dist"], metric=u["metric"])

#         for h in hdbscan_params:
#             hdbscan_model = hdbscan.HDBSCAN(
#                 min_cluster_size=h["min_cluster_size"],
#                 min_samples=h["min_samples"],
#                 cluster_selection_epsilon=h["cluster_selection_epsilon"]
#             )

#             for b in bertopic_params:
#                 topic_model = BERTopic(
#                     embedding_model=embedding_model,
#                     umap_model=umap_model,
#                     hdbscan_model=hdbscan_model,
#                     top_n_words=b["top_n_words"],
#                     nr_topics=b["nr_topics"],
#                     verbose=False
#                 )

#                 topics, probs = topic_model.fit_transform(texts_pos)
#                 print(f"\nModel config: {emb}, UMAP: {u}, HDBSCAN: {h}, BERTopic: {b}")
#                 evaluate_topics(texts_pos, topic_model)

#                 # Log topic count and topic list
#                 all_topics = topic_model.get_topics()
#                 topics_filtered = {k: v for k, v in all_topics.items() if k != -1 and v is not None}

#                 print(f"🔢 Number of topics (excluding -1): {len(topics_filtered)}")
#                 for topic_id, words in topics_filtered.items():
#                     word_list = [word for word, _ in words[:10]]
#                     print(f"🧠 Topic {topic_id}: {', '.join(word_list)}")


Model config: paraphrase-multilingual-MiniLM-L12-v2, UMAP: {'n_neighbors': 15, 'min_dist': 0.0, 'metric': 'cosine'}, HDBSCAN: {'min_cluster_size': 5, 'min_samples': 1, 'cluster_selection_epsilon': 0.0}, BERTopic: {'top_n_words': 10, 'nr_topics': None}
Coherence (c_v): 0.35
Coherence (u_mass): -11.812
Coherence (c_uci): -7.859
Coherence (c_npmi): -0.176
IRBO Topic Diversity: 0.974
🔢 Number of topics (excluding -1): 23
🧠 Topic 0: teman, kuliah, kampus, pertemanan, dosen, lokasi, hal, berinteraksi, dipertemukan, terbentuk
🧠 Topic 1: lingkungan, kampus, aktif, mendukung, nyaman, perkuliahan, akademik, belajar, terkhusus, pelajaran
🧠 Topic 2: kegiatan, bermain, teman, muda, bareng, perkuliahan, senang, main, fun, menyenangkan
🧠 Topic 3: materi, dosen, mengajar, praktek, sistem, mengajarkan, binusmaya, mengutamakan, mengakses, kelas
🧠 Topic 4: koneksi, kawan, teman, dapatkan, memperbanyak, kisi, bertemu, bebas, seru, membantu
🧠 Topic 5: online, fasilitas, pembelajaran, pengalaman, dosen, pr


Model config: indobenchmark/indobert-base-p1, UMAP: {'n_neighbors': 15, 'min_dist': 0.0, 'metric': 'cosine'}, HDBSCAN: {'min_cluster_size': 5, 'min_samples': 1, 'cluster_selection_epsilon': 0.0}, BERTopic: {'top_n_words': 10, 'nr_topics': None}
Coherence (c_v): 0.326
Coherence (u_mass): -12.828
Coherence (c_uci): -8.394
Coherence (c_npmi): -0.171
IRBO Topic Diversity: 0.989
🔢 Number of topics (excluding -1): 33
🧠 Topic 0: lingkungan, teman, stres, pertemanan, suportif, dipertemukan, keluarga, belajar, kuliah, perkuliahan
🧠 Topic 1: mengajar, dosen, mudah, materi, sistem, murid, diajak, dipahami, besar, terstruktur
🧠 Topic 2: bareng, kawan, seru, teman, temen, berinteraksi, asik, suasana, kakak, kampus
🧠 Topic 3: mata, kuliah, bagus, science, magang, data, program, mulut, computer, no
🧠 Topic 4: pengalaman, kerja, insight, mengharuskan, mahasiswa, magang, kebutuhan, binus, berguna, program
🧠 Topic 5: matkul, materi, teman, muda, dosen, enjoy, mudah, dipahami, asik, terstruktur
🧠 Topic 

In [ ]:
# from sentence_transformers import SentenceTransformer
# from bertopic import BERTopic
# from umap import UMAP
# import hdbscan

# for emb in embedding_models:
#     embedding_model = SentenceTransformer(emb)

#     for u in umap_params:
#         umap_model = UMAP(n_neighbors=u["n_neighbors"], min_dist=u["min_dist"], metric=u["metric"])

#         for h in hdbscan_params:
#             hdbscan_model = hdbscan.HDBSCAN(
#                 min_cluster_size=h["min_cluster_size"],
#                 min_samples=h["min_samples"],
#                 cluster_selection_epsilon=h["cluster_selection_epsilon"]
#             )

#             for b in bertopic_params:
#                 topic_model = BERTopic(
#                     embedding_model=embedding_model,
#                     umap_model=umap_model,
#                     hdbscan_model=hdbscan_model,
#                     top_n_words=b["top_n_words"],
#                     nr_topics=b["nr_topics"],
#                     verbose=False
#                 )

#                 topics, probs = topic_model.fit_transform(texts_neg)
#                 print(f"\nModel config: {emb}, UMAP: {u}, HDBSCAN: {h}, BERTopic: {b}")
#                 evaluate_topics(texts_neg, topic_model)

#                 # Log topic count and topic list
#                 all_topics = topic_model.get_topics()
#                 topics_filtered = {k: v for k, v in all_topics.items() if k != -1 and v is not None}

#                 print(f"🔢 Number of topics (excluding -1): {len(topics_filtered)}")
#                 for topic_id, words in topics_filtered.items():
#                     word_list = [word for word, _ in words[:10]]
#                     print(f"🧠 Topic {topic_id}: {', '.join(word_list)}")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]


Model config: indobenchmark/indobert-base-p1, UMAP: {'n_neighbors': 15, 'min_dist': 0.0, 'metric': 'cosine'}, HDBSCAN: {'min_cluster_size': 5, 'min_samples': 1, 'cluster_selection_epsilon': 0.0}, BERTopic: {'top_n_words': 10, 'nr_topics': None}
Coherence (c_v): 0.35
Coherence (u_mass): -12.389
Coherence (c_uci): -8.072
Coherence (c_npmi): -0.172
IRBO Topic Diversity: 0.981
🔢 Number of topics (excluding -1): 26
🧠 Topic 0: kampus, lambat, lantai, itb, lcd, seringkali, error, tersedia, gedung, lift
🧠 Topic 1: mengajar, dosen, mengerti, mengajarkan, teori, bersemangat, dimengerti, materi, kompeten, mahasiswa
🧠 Topic 2: materi, kelompok, dosen, kerja, nilai, pengajaran, penjelasan, metode, pembelajaran, interaksi
🧠 Topic 3: materi, dunia, praktikum, pembelajaran, niat, kurikulum, perkuliahan, dosen, abstrak, bermanfaat
🧠 Topic 4: kantin, area, diperbaiki, parkir, fasilitas, dibandingkan, lahan, makan, harga, kampus
🧠 Topic 5: koneksi, jaringan, wifi, perkembangan, kesulitan, online, kendal

# Test Alice

In [ ]:
# embedding_models = [
#     "paraphrase-multilingual-MiniLM-L12-v2",
#     "indobenchmark/indobert-base-p1"
# ]

# umap_params = [
#     {"n_neighbors": 15, "min_dist": 0.0, "metric": "cosine"},
#     {"n_neighbors": 10, "min_dist": 0.1, "metric": "euclidean"}
# ]

# hdbscan_params = [
#     {"min_cluster_size": 5, "min_samples": 1, "cluster_selection_epsilon": 0.0},
#     {"min_cluster_size": 15, "min_samples": 2, "cluster_selection_epsilon": 0.5}
# ]

# bertopic_params = [
#     {"top_n_words": 10, "nr_topics": None},
#     {"top_n_words": 8, "nr_topics": 10}
# ]


In [ ]:
# from sentence_transformers import SentenceTransformer
# from bertopic import BERTopic
# from umap import UMAP
# import hdbscan

# for emb in embedding_models:
#     embedding_model = SentenceTransformer(emb)

#     for u in umap_params:
#         umap_model = UMAP(n_neighbors=u["n_neighbors"], min_dist=u["min_dist"], metric=u["metric"])

#         for h in hdbscan_params:
#             hdbscan_model = hdbscan.HDBSCAN(
#                 min_cluster_size=h["min_cluster_size"],
#                 min_samples=h["min_samples"],
#                 cluster_selection_epsilon=h["cluster_selection_epsilon"]
#             )

#             for b in bertopic_params:
#                 topic_model = BERTopic(
#                     embedding_model=embedding_model,
#                     umap_model=umap_model,
#                     hdbscan_model=hdbscan_model,
#                     top_n_words=b["top_n_words"],
#                     nr_topics=b["nr_topics"],
#                     verbose=False
#                 )

#                 topics, probs = topic_model.fit_transform(texts_pos)
#                 print(f"\nModel config: \nEmbedding: {emb} \nUMAP: {u} \nHDBSCAN: {h} \nBERTopic: {b}")
#                 evaluate_topics(texts_pos, topic_model)

#                 # Log topic count and topic list
#                 all_topics = topic_model.get_topics()
#                 topics_filtered = {k: v for k, v in all_topics.items() if k != -1 and v is not None}

#                 print(f"🔢 Number of topics (excluding -1): {len(topics_filtered)}")
#                 for topic_id, words in topics_filtered.items():
#                     word_list = [word for word, _ in words[:10]]
#                     print(f"🧠 Topic {topic_id}: {', '.join(word_list)}")